In [1]:
import os
import sys
import math
import json
import torch
import PIL
import numpy as np
from tqdm import tqdm

import scipy.io
from scipy import ndimage
import matplotlib
# from skimage import io
# matplotlib.use("pgf")
matplotlib.rcParams.update({
    # 'font.family': 'serif',
    'font.size':8,
})
from matplotlib import pyplot as plt

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import TensorBoardLogger
seed_everything(42)

import DiffNet
from DiffNet.DiffNetFEM import DiffNet2DFEM
from torch.utils import data
from DiffNet.networks.autoencoders import AE
# from e1_stokes_base_resmin import Stokes2D

from pytorch_lightning.callbacks.base import Callback

torch.set_printoptions(precision=10)

from e2_ns_fpc_embedded_airfoil import NS_FPC_Dataset, NS_FPC, OptimSwitchLBFGS

/work/baskarg/bkhara/python_virtual_envs/lightning/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
Global seed set to 42
Global seed set to 42


In [2]:
# image_dataset_dir = "../AirfoilFPC-256x128"
image_dataset_dir = "../test-dataset-fpc"
# mapping_type = 'no_network'
mapping_type = 'network'
lx = 5.
ly = 12.
Nx = 256
Ny = 128
domain_size = 128
Re = 1.
dir_string = "ns_fpc_af"
max_epochs = 50001
plot_frequency = 1
LR = 2e-4
opt_switch_epochs = max_epochs
load_from_prev = False
load_version_id = 37

enable_progress_bar = True
# print("argv = ", sys.argv)
# if len(sys.argv) > 1:
#     enable_progress_bar = bool(int(sys.argv[1]))
#     print("enable_progress_bar = ", enable_progress_bar)

x = np.linspace(0, 1, Nx)
y = np.linspace(0, 1, Ny)
xx , yy = np.meshgrid(x, y)

dataset = NS_FPC_Dataset(dirname=image_dataset_dir, domain_lengths=(lx,ly), domain_sizes=(Nx,Ny), Re=Re)
if load_from_prev:
    print("LOADING FROM PREVIOUS VERSION: ", load_version_id)
    case_dir = './ns_fpc_af/version_'+str(load_version_id)
    net_u = torch.load(os.path.join(case_dir, 'net_u.pt'))
    net_v = torch.load(os.path.join(case_dir, 'net_v.pt'))
    net_p = torch.load(os.path.join(case_dir, 'net_p.pt'))
else:
    print("INITIALIZING PARAMETERS TO ZERO")
    if mapping_type == 'no_network':
        v1 = np.zeros_like(dataset.x)
        v2 = np.zeros_like(dataset.x)
        p  = np.zeros_like(dataset.x)
        u_tensor = np.expand_dims(np.array([v1,v2,p]),0)

        # network = torch.nn.ParameterList([torch.nn.Parameter(torch.FloatTensor(u_tensor), requires_grad=True)])
        net_u = torch.nn.ParameterList([torch.nn.Parameter(torch.FloatTensor(u_tensor[:,0:1,:,:]), requires_grad=True)])
        net_v = torch.nn.ParameterList([torch.nn.Parameter(torch.FloatTensor(u_tensor[:,1:2,:,:]), requires_grad=True)])
        net_p = torch.nn.ParameterList([torch.nn.Parameter(torch.FloatTensor(u_tensor[:,2:3,:,:]), requires_grad=True)])
    elif mapping_type == 'network':
        net_u = AE(in_channels=1, out_channels=1, dims=Ny, n_downsample=3)
        net_v = AE(in_channels=1, out_channels=1, dims=Ny, n_downsample=3)
        net_p = AE(in_channels=1, out_channels=1, dims=Ny, n_downsample=3)
network = (net_u, net_v, net_p)
basecase = NS_FPC(network, dataset, domain_lengths=(lx,ly), domain_sizes=(Nx,Ny), batch_size=32, fem_basis_deg=1, learning_rate=LR, plot_frequency=plot_frequency, mapping_type=mapping_type)


100%|██████████| 8/8 [00:00<00:00, 497.78it/s]

Initializing parametric dataset
dataset size =  (8, 6, 128, 256)
INITIALIZING PARAMETERS TO ZERO


In [3]:
# Initialize trainer
logger = pl.loggers.TensorBoardLogger('.', name=dir_string)
csv_logger = pl.loggers.CSVLogger(logger.save_dir, name=logger.name, version=logger.version)

early_stopping = pl.callbacks.early_stopping.EarlyStopping('loss_u',
    min_delta=1e-8, patience=10, verbose=False, mode='max', strict=True)
checkpoint = pl.callbacks.model_checkpoint.ModelCheckpoint(monitor='loss',
    dirpath=logger.log_dir, filename='{epoch}-{step}',
    mode='min', save_last=True)

lbfgs_switch = OptimSwitchLBFGS(epochs=opt_switch_epochs)

trainer = Trainer(gpus=[0],callbacks=[early_stopping,lbfgs_switch],
    checkpoint_callback=checkpoint, logger=[logger,csv_logger],
    max_epochs=max_epochs, deterministic=True, profiler="simple")

# Training
trainer.fit(basecase)
# Save network
torch.save(basecase.net_u, os.path.join(logger.log_dir, 'net_u_detached.pt'))
torch.save(basecase.net_v, os.path.join(logger.log_dir, 'net_v_detached.pt'))
torch.save(basecase.net_p, os.path.join(logger.log_dir, 'net_p_detached.pt'))

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/work/baskarg/bkhara/python_virtual_envs/lightning/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Experiment logs directory ./ns_fpc_af/version_16 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  warnings.warn(*args, **kwargs)

   | Name         | Type          | Params
------------------------------------------------
0  | network      | AE            | 42.0 M
1  | N_gp         | ParameterList | 16    
2  | dN_x_gp      | ParameterList | 16    
3  | dN_y_gp      | ParameterList | 16    
4  | d2N_x_gp     | ParameterList | 16    
5  | d2N_y_gp     | ParameterList | 16    
6  | d2N_xy_gp    | ParameterList | 16    
7  | N_gp_surf    | ParameterList | 4     
8  | dN_x_gp_surf | ParameterList | 4     
9  | dN_y_gp_surf | ParameterList | 4     
10 | net_v        | AE      

Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s] 

/work/baskarg/bkhara/python_virtual_envs/lightning/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Epoch 8: 100%|██████████| 1/1 [00:00<00:00,  2.43it/s, loss=572, v_num=6_16]     

/work/baskarg/bkhara/python_virtual_envs/lightning/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
